<a href="https://colab.research.google.com/github/yf591/Using-Stable-Diffusion-in-Google-Colab/blob/main/Image_Generation_AI_(Stable_Diffusion).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font color=fbb03b>事前準備
- ライブラリのインストール
- ライブラリインポート

In [ ]:
# 必要なライブラリをインストール
!pip install diffusers transformers peft

**解説**: diffusers, transformers, peft の3つのライブラリをインストールします。これにより、DiffusionPipeline（拡散モデル）や他の機械学習モデルを使用できるようになります。

- diffusers: 拡散モデルを実装したライブラリ。
- transformers: トランスフォーマーモデル（BERTやGPTなど）を扱うためのライブラリ。
- peft: パラメータ効率の良いファインチューニング（PEFT）を行うためのライブラリ。

In [20]:
# 必要なライブラリをインポート
import torch  # PyTorchのライブラリをインポート

# 必要に応じてDiffusionPipelineまたはStableDiffusionPipelineをインポート
from diffusers import DiffusionPipeline
from diffusers import StableDiffusionPipeline

import matplotlib.pyplot as plt # 可視化のためのライブラリをインポート

**解説**: torchはPyTorchのコアライブラリをインポートし、DiffusionPipelineは拡散モデルを簡単に利用できるようにするためのクラスを提供しています。

**torch**: ディープラーニングのフレームワーク。

**DiffusionPipeline**: テキストや画像生成のための拡散モデルパイプラインを提供するクラス。

- 説明: DiffusionPipelineは、一般的な拡散モデルを扱うためのベースクラスです。このクラスは、さまざまな拡散モデル（Stable Diffusionを含む）に対して使用できますが、特定のモデル向けの最適化はされていない場合があります。
- 主な機能:
  - さまざまな拡散モデルを利用するための汎用的な機能を提供します。
  - モデルを選択する柔軟性があるため、異なる拡散技術を使いたい場合に便利です。

**StableDiffusionPipeline**: 高品質な画像生成のためのパイプラインを提供するクラス。テキストプロンプトを基にリアルな画像を生成する機能を持つ。
- 説明: StableDiffusionPipelineは、Stable Diffusionモデル専用のパイプラインクラスです。これを使用すると、Stable Diffusionに特化した設定やオプションを簡単に利用できます。
- 主な機能:
  - 特定のプロンプトに基づいて画像を生成します。
  - モデルの設定がStable Diffusion向けに最適化されているため、使いやすさが向上します。

以下のセルを実行してドライブをマウントし実行ディレクトリへ移動します.

In [ ]:
# Google Drive をマウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 以下はサンプルです.ご自身の環境に合わせてフォルダへのパスは変更してください.
%cd /content/drive/My Drive/Colab Notebooks/【画像生成】Stable Dffusion

#<font color=fbb03b>使用するモデルについて
- 使用するモデルの指定
- モデルを学習済みのものから読み込み

## 1. DiffusionPipelineを使用

In [5]:
model_id = 'gsdf/Counterfeit-V2.5'  # 使用するモデルIDを指定
pipe = DiffusionPipeline.from_pretrained(model_id)  # モデルを事前学習済みのものから読み込む

- Hugging Faceモデルハブで公開されているモデルのIDを指定します。ここでは例として'gsdf/Counterfeit-V2.5'という名前のモデルを使用しています。
 - モデルID: Hugging Faceで管理されている拡散モデルの識別子。

- from_pretrainedメソッドを使って、指定したmodel_idの事前学習済みモデルをHugging Faceのリポジトリから読み込みます。
 - from_pretrained: 指定したモデルを事前に学習した状態でロードする関数。

## 2. StableDiffusionPipelineを使用

In [26]:
# Stable Diffusionを使用して高品質なリアルな画像を生成するための一般的なモデル
from huggingface_hub import login
login("Hugging_FaceのTokenを入力") # Hugging Faceにログイン（Access TokensのValueを入力）

model_id = "CompVis/stable-diffusion-v1-4" # 使用するモデルを入力
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

### StableDiffusionPipeline使用の補足説明
以下の3パターンのコードは、`StableDiffusionPipeline` を初期化する方法ですが、それぞれ異なるオプションが指定されています。これらの違いを説明します。

### 1. `pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")`
- **torch_dtype=torch.float16**: このオプションは、モデルの重みを16ビット浮動小数点（`float16`）形式で読み込むことを指定します。これにより、メモリ使用量を削減し、GPU上での計算を高速化することが可能です。
- **.to("cuda")**: モデルをGPUに転送するためのメソッドです。これにより、GPUを利用してより高速な計算を行うことができます。特に、画像生成タスクでは処理速度が向上します。

### 2. `pipe = StableDiffusionPipeline.from_pretrained(model_id)`
- **デフォルト設定**: こちらのコードは、特にオプションを指定せずにモデルを読み込む方法です。デフォルトでは、モデルは32ビット浮動小数点（`float32`）形式でロードされ、CPUまたはGPUに明示的に転送されることはありません。
- **GPUに自動転送されない**: 明示的に`.to("cuda")`を指定しない限り、モデルはCPU上で実行されます。

### 3. `pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=True)`
- **use_auth_token=True**: このオプションは、Hugging Face Hubに対する認証トークンを使用して、プライベートモデルやアクセス制限のあるモデルを取得することを指定します。このトークンが必要な場合に、正しい認証が行われるようにします。
- **その他の設定はデフォルト**: モデルはデフォルトの32ビット浮動小数点形式で読み込まれ、CPU上で実行されます（GPUに移動する場合は `.to("cuda")` が必要）。

### まとめ
- **メモリと速度**: `torch_dtype=torch.float16` は、メモリ効率と速度向上を図るためのオプションです。
- **GPUの使用**: `.to("cuda")` を使用すると、計算速度が向上しますが、デフォルトの設定ではCPUで実行されます。
- **認証**: `use_auth_token=True` を指定することで、プライベートモデルへのアクセスが可能になります。

これにより、各パターンは異なる使用目的や実行環境に応じて最適化されています。

#<font color=fbb03b>画像の生成
- 入力するプロンプトを定義
- 生成した画像の取得

In [40]:
# 入力するプロンプトを定義
prompt = "ここにプロンプトを入力" # プロンプトを入力して定義する

**解説**: 生成したい画像を説明するテキスト（プロンプト）を入力します。このプロンプトに基づいて拡散モデルが画像を生成します。
- プロンプト: モデルが生成するための指示（テキスト）。

## 一枚の画像を生成する方法

In [ ]:
# 一枚の画像を生成
img = pipe(prompt).images[0] # プロンプトを使って画像を生成し、その画像を取得

In [ ]:
# 画像を取得して表示
plt.imshow(img)
plt.axis('off')  # 軸を非表示にする
plt.show()  # 画像をColab上で表示

**解説**: pipeにプロンプトを渡して画像生成を行います。上記の例では生成された1枚の画像を取得しています。
- pipe(prompt): プロンプトに基づいて拡散モデルが画像を生成。
- .images[0]: 生成された画像リストの中から画像を取り出す。

## 生成枚数を指定して複数の画像を生成する方法

In [ ]:
# 画像の枚数を指定
num_images = 5 # 生成する画像の数を指定
images = pipe(prompt, num_images_per_prompt=num_images).images[:] # プロンプトを使って画像を生成し、取得する画像を指定

In [ ]:
# 画像を取得して表示
for i, img in enumerate(images[:]):
    plt.imshow(img)
    plt.axis('off')  # 軸を非表示にする
    plt.show()  # 画像をColab上で表示

**解説**: pipeにプロンプトを渡して画像生成を行います。生成された複数の画像のうち、指定された画像を取得しています。
- pipe(prompt, num_images_per_prompt=num_images): プロンプトに基づいて拡散モデルが画像を生成。
  - num_images_per_promptを使って、必要な画像の枚数を指定します。たとえば、num_images_per_prompt = 10と設定すると、10枚の画像が生成されます。
- .images[:5]: 生成された画像リストの中から1から5枚目（listの0~4番目）の画像を取り出す。
  - Pythonのリストでは、list[start:end]の形式でスライスが可能です。この場合、最初の5枚を取得するにはimages[:5]と書きます。

#<font color=fbb03b>生成した画像の保存

## 一枚の画像を生成する方法で取得した画像の保存

In [ ]:
# img.save("ファイル名.jpg") # 生成された画像を指定したファイル名で保存
img.save("/content/drive/MyDrive//Colab Notebooks/【画像生成】Stable Dffusion/ファイル名.jpg") # ファイル名を入力してその画像ファイルをGoogle Driveに保存

**解説**: 生成した画像を「ファイル名（ここにファイル名を入力する）.jpg」として保存します。ここで指定したファイル名で保存先が決まります。
- img.save: 画像を指定された名前のファイルとして保存するメソッド。

## 画像の枚数を指定する方法で取得した画像の保存

In [ ]:
# 1～3枚目(listの0,1,2)の画像を取得
my_images = images[:3]

# それぞれの画像を保存
for i, img in enumerate(my_images):
    img.save(f"/content/drive/MyDrive//Colab Notebooks/【画像生成】Stable Dffusion/image_{i+1}.jpg")  # "image_1.jpg", "image_2.jpg", "image_3.jpg" というファイル名で保存

**解説**:

このコードでは、生成された画像を my_images にスライスで格納しています。それぞれの画像は image_1.jpg、image_2.jpg、image_3.jpg という名前で保存されます。